This notebook assumes you have run:

```
git clone https://dagshub.com/onefact/data.payless.health.git
cd data.payless.health
dvc pull
```


In [10]:
import pandas as pd
import wget 
import os
from urllib.parse import urlparse
from pathlib import Path
raw_data_dir = Path("/Users/me/tmp/data.payless.health/data/raw")

In [11]:
df = pd.read_csv(raw_data_dir / "220106-dolthub.com-onefact-paylesshealth-hospitals.csv")

In [12]:
df.head()

,ccn,name,state,city,homepage,cdm_indirect_url,cdm_url
0,154063,"NEUROPSYCHIATRIC HOSPITAL OF INDIANAPOLIS, LLC",IN,INDIANAPOLIS,https://www.neuropsychiatrichospitals.net/,https://www.neuropsychiatrichospitals.net/pric...,https://www.neuropsychiatrichospitals.net/wp-c...
1,154064,ASSURANCE HEALTH PSYCHIATRIC HOSPITAL,IN,INDIANAPOLIS,http://assurancehealthsystem.com/,NaN,NaN
2,154065,NEURO BEHAVIORAL HOSPITAL,IN,CROWN POINT,https://www.neuropsychiatrichospitals.net/,https://www.neuropsychiatrichospitals.net/pric...,https://www.neuropsychiatrichospitals.net/wp-c...
3,154066,BRIGHTWELL BEHAVIORAL HEALTH,IN,CLARKSVILLE,https://brightwellbehavioral.com/,NaN,NaN
4,154067,HENDRICKS BEHAVIORAL HOSPITAL,IN,PLAINFIELD,https://www.hendricksbehavioralhospital.com/,https://www.hendricksbehavioralhospital.com/,https://s3.amazonaws.com/ycubaa-production-mar...


In [34]:

error_rows = []
for _, df_row in df.iterrows():
    try:
        filename = os.path.basename(urlparse(df_row["cdm_url"]).path)
        out_path = raw_data_dir / f"{df_row['ccn']}_{filename}"
        print(out_path)
        wget.download(df_row["cdm_url"], out=str(out_path))
    except:
        error_rows.append(df_row)

/Users/me/tmp/data.payless.health/data/raw/154063_NPH-Price-Tranparency-Pricing.xlsx
/Users/me/tmp/data.payless.health/data/raw/154065_NPH-Price-Tranparency-Pricing.xlsx
/Users/me/tmp/data.payless.health/data/raw/154067_85-2580772_HendricksBehavioralHealth_Standardcharges_11072022.xlsx
/Users/me/tmp/data.payless.health/data/raw/159801_351963748_Lutheran_standardcharges.csv
/Users/me/tmp/data.payless.health/data/raw/159802_350869066_ascension-st-vincent-hospital_standardcharges.xlsx
/Users/me/tmp/data.payless.health/data/raw/159803_35-1720796_BLOOMINGTON_HOSPITAL_standardcharges22.csv


TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'